# Run the x-match again, but now using the code in 'booq'

As a final stage, the code used in the past notebooks was transfered to ``booq``. Now we have to test it, looking for some erroneous reference or whatever bug eventualy to show up.


In [1]:
import booq

In [2]:
# let's define some parameters for our mock catalogs

# to be real fake, we have to fake evrything... :(
# let's start with the size of the supposed images
# from where the sources are coming from...
sx = 5000
sy = 5000

# number of sources to be relative to the size of the image
num_sources = int( 0.1 * (sx*sy)/(sx+sy) )

# and, say there is an error radius
error_radius = 20

In [3]:
from booq.data import mock
from booq import coordinates

# Generate catalog A
A_image_catalog = mock.D2.generate_catalog(num_sources, error_radius, (sy,sx))

A = A_image_catalog.rename(columns={'ID':'id'})
A.loc[:,'ra']  = coordinates.pixel_to_sky(A_image_catalog.x, 0, 1.0/sx)
A.loc[:,'dec'] = coordinates.pixel_to_sky(A_image_catalog.y, 0, 1.0/sy)

# Generate catalog B
B_image_catalog = mock.D2.generate_catalog_around(A_image_catalog, 0.75, (sy,sx))

B = B_image_catalog.rename(columns={'ID':'id'})
B.loc[:,'ra']  = coordinates.pixel_to_sky(B_image_catalog.x, 0, 1.0/sx)
B.loc[:,'dec'] = coordinates.pixel_to_sky(B_image_catalog.y, 0, 1.0/sy)


In [4]:
from booq.pipelines import xmatch_nn
# X-match the catalogs
matched_catalog_serial = xmatch_nn.xmatch(A, B)

/home/chbrandt/fido/repos/booq/booq/pipelines/xmatch_nn.py:121: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  B_matched['A_idx'] = df_matched_idx.index


In [5]:
from booq.pipelines import xmatch_nn
# X-match the catalogs
matched_catalog_parallel = xmatch_nn.xmatch(A, B, parallel_run=True)

Running in parallel
Running in parallel


In [6]:
matched_catalog_parallel.equals(matched_catalog_serial)

True

In [7]:
from booq.table.atable import aTable
aTable.from_pandas(matched_catalog_parallel).show_in_notebook()

A_id,A_r,A_x,A_y,A_ra,A_dec,B_id,B_r,B_x,B_y,B_ra,B_dec,AB_dist
1.0,20.0,2249.0,146.0,0.4498,0.0292,1.0,20.0,2232.0,120.0,0.4464,0.024,22.3664026523
2.0,20.0,4509.0,2379.0,0.9018,0.4758,2.0,20.0,4503.0,2404.0,0.9006,0.4808,18.5111074623
3.0,20.0,1326.0,3287.0,0.2652,0.6574,3.0,20.0,1333.0,3324.0,0.2666,0.6648,27.1125029884
4.0,20.0,1449.0,100.0,0.2898,0.02,4.0,20.0,1442.0,82.0,0.2884,0.0164,13.9055096069
5.0,20.0,1401.0,4706.0,0.2802,0.9412,5.0,20.0,1401.0,4689.0,0.2802,0.9378,12.24
6.0,20.0,257.0,2603.0,0.0514,0.5206,6.0,20.0,265.0,2581.0,0.053,0.5162,16.8546873023
7.0,20.0,4474.0,0.0,0.8948,0.0,nan,nan,nan,nan,nan,nan,nan
8.0,20.0,3148.0,4217.0,0.6296,0.8434,8.0,20.0,3149.0,4219.0,0.6298,0.8438,1.60993404434
9.0,20.0,2338.0,763.0,0.4676,0.1526,9.0,20.0,2356.0,781.0,0.4712,0.1562,18.3281744926
10.0,20.0,2293.0,3523.0,0.4586,0.7046,10.0,20.0,2284.0,3508.0,0.4568,0.7016,12.5946050783
